# **S6_Send Emails**

# **Part 1 Copy Files**

In [1]:
#define function
import os
import shutil
date_input=input('Enter the date frame as mmdd-mmdd: ')
destination_folder_path=fr'C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Mail Merge Macro\{date_input}'

def Copy_File(source_folder_path,destination_folder_path):
    os.makedirs(destination_folder_path, exist_ok=True)
    for filename in os.listdir(source_folder_path):
        source_file_path = os.path.join(source_folder_path,filename)
        if os.path.isfile(source_file_path):#move files only, do not move folder
            destination_file_path = os.path.join(destination_folder_path, filename)
            shutil.copy(source_file_path,destination_file_path)

Enter the date frame as mmdd-mmdd:  1223-1229


In [2]:
#move patient alerts files to email folder
Copy_File(fr'C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} PBACO',destination_folder_path) 
Copy_File(fr'C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} PBACO\patient_alerts_{date_input}',destination_folder_path) #all files for individual pc

Copy_File(fr'C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} NHNP',destination_folder_path)
Copy_File(fr'C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} NHNP\patient_alerts_NHNP_{date_input}',destination_folder_path)

Copy_File(fr'C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} Empire',destination_folder_path)
Copy_File(fr'C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} Empire\patient_alerts_Empire_{date_input}',destination_folder_path)

#move App usage Overview
shutil.copy(fr'R:\Patient Alerts By Subscribed NPI\2024 App Performance PBACO.xlsx',destination_folder_path)
shutil.copy(fr'R:\Patient Alerts By Subscribed NPI - NHNP\2024 App Performance NHNP.xlsx',destination_folder_path)
shutil.copy(fr'R:\Patient Alerts By Subscribed NPI - Empire\2024 App Performance Empire.xlsx',destination_folder_path)

#move Escalation Policy
shutil.copy(fr'C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Mail Merge Macro\Escalation Policy Draft_SuperDOC ACO App_JB_v.3.pdf',destination_folder_path)

#move TCM_PDV
shutil.copy(fr'C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\TCM_PDV Audit Tracker 2024.xlsx',destination_folder_path)

'C:\\Users\\ZemingLiu\\OneDrive - PBACO\\Documents\\Weekly App Reports\\Derrell Macros\\Mail Merge Macro\\1223-1229\\TCM_PDV Audit Tracker 2024.xlsx'

# **Part 2 Send Emails**

In [3]:
import win32com.client as win32
import time
from win32clipboard import OpenClipboard, GetClipboardData, CloseClipboard

#open excel email workbook
excel_app = win32.Dispatch('Excel.Application')
excel_app.Visible = True
wb = excel_app.Workbooks.Open(r'C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Mail Merge Macro\Derrell Mail Merge Macro.xlsm')
outlook_app = win32.Dispatch('Outlook.Application')

In [54]:
def create_outlook_email(i): 
    #template
    template_path = fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Mail Merge Macro\{ws.Range(f'C{i}').Value}"
    email = outlook_app.CreateItemFromTemplate(template_path)
    
    #modify the email
    email.Subject = ws.Range(f'G{i}').Value
    email.Body = email.Body.replace("[date_range]", date_input)
    email.Body = email.Body.replace("[date_label]", date_input)
    email.To = ws.Range(f'D{i}').Value
    email.CC=ws.Range(f'F{i}').Value if ws.Range(f'F{i}').Value else "" 
    
    #define visible rows function
    def get_visible_rows(worksheet):
        visible_rows = []
        used_range = worksheet.UsedRange
        for row in used_range.Rows:
            if not row.Hidden and row.Cells(1, 1).Value is not None:  # Check if the row is visible and not empty
                visible_rows.append(row)
        return visible_rows
    
    #attachment
    filenames=ws.Range(fr'I{i}').Value.split(';')
    for filename in filenames:
        filename=filename.strip()
        if ws.Name == 'PBACO_LowApp': #for low performer email, copy and paste into the email
            lowperformers_wb=excel_app.Workbooks.Open(fr'C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Mail Merge Macro\{date_input}\{filename}')
            lowperformers_ws=lowperformers_wb.Sheets(f'{date_input}')
            
            #add filters
            if lowperformers_ws.AutoFilterMode:# Clear existing filters
                lowperformers_ws.AutoFilterMode = False 
            lowperformers_ws.Range(  # Apply filter
                lowperformers_ws.Cells(1, 1),
                lowperformers_ws.Cells(1, lowperformers_ws.UsedRange.Columns.Count)
            ).AutoFilter(
                Field=5,  # Provider Consultant
                Criteria1=ws.Range(f'B{i}').Value
            )
            
            #check if skip pc
            visible_rows = get_visible_rows(lowperformers_ws)
            print(len(visible_rows))
            if len(visible_rows) <= 1:
                print("No data available to copy, skip this PC")
                return     
            else:
                #copy the data and paste to email body
                visible_data = lowperformers_ws.UsedRange.SpecialCells(12)
                visible_data.Copy()
                time.sleep(1) 
                word_editor = email.GetInspector.WordEditor
                paragraphs = word_editor.Paragraphs
                paragraphs[1].Range.Select()
                word_editor.Application.Selection.Paste() # Paste content
                excel_app.CutCopyMode = False
                email.Display()
            lowperformers_wb.Close(SaveChanges=0)
        else:
            email.Attachments.Add(fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Mail Merge Macro\{date_input}\{filename}")
            email.Display()

In [5]:
#Patient Alerts
ws=wb.Sheets('PBACO_PAlert') #initialize the list to generate email
ws.Range('H2').Value=date_input
for i in range(2,ws.UsedRange.Rows.Count):
    print(fr"Processing {ws.Range(f'A{i}').Value}")
    create_outlook_email(i)

Processing Alexis
Processing Amie
Processing Andres D
Processing Andres
Processing Brian
Processing Christina
Processing Cynthia
Processing Daimarys
Processing David T
Processing Deanna
Processing Drew
Processing Diamarys
Processing Emily
Processing Fabian
Processing George
Processing Gianfranco
Processing Gisselle
Processing Heidi
Processing Holly G
Processing Jamilin
Processing Karina
Processing Kendra
Processing Katherine
Processing Katiann
Processing Keith
Processing Matt
Processing Mackenzie
Processing Monti
Processing Natalie
Processing Ondina 
Processing Parath
Processing Rachel
Processing Rejane
Processing Sharada
Processing Trent
Processing Gianna + Jolea (send their PBACO and NHNP files together in the same email as they work together)
Processing Callie
Processing Nancy (send PBACO and NHNP file in same email)
Processing Nathalia
Processing Shymara
Processing Lindy
Processing Tiffany (send PBACO and Empire file in same email)
Processing Wadha
Processing Susan


In [55]:
#Low Performers
ws=wb.Sheets('PBACO_LowApp') #choose which list to generate email
ws.Range('H2').Value=date_input
for i in range(2,ws.UsedRange.Rows.Count):
    print(fr"Processing {ws.Range(f'A{i}').Value}")
    create_outlook_email(i)

Processing Alexis
1
No data available to copy, skip this PC
Processing Amie
8
Processing Andres D
4
Processing Andres
4
Processing Brian
7
Processing Christina
4
Processing Cynthia
4
Processing Daimarys
9
Processing Diamarys
15
Processing Drew
7
Processing Fabian
1
No data available to copy, skip this PC
Processing George
9
Processing Gianfranco
9
Processing Nicole
8
Processing Gianna + Jolea 
(send their PBACO and  NHNP files together in the same email as they work together)
6
Processing Gianna + Jolea 
(send their PBACO, AHP, NHNP files together in the same email as they work together)
29
Processing Gisselle
12
Processing Heidi
9
Processing Holly G
1
No data available to copy, skip this PC
Processing Karina
1
No data available to copy, skip this PC
Processing Kendra
11
Processing Katherine
7
Processing Katiann
27
Processing Matt
5
Processing Mackenzie
4
Processing Monti
16
Processing Natalie
10
Processing Ondina
10
Processing Parath
5
Processing Jamilin
5
Processing Keith
1
No data a

In [56]:
#Leadership
ws=wb.Sheets('leadership') #choose which list to generate email
ws.Range('H2').Value=date_input
for i in range(2,5):
    print(fr"Processing {ws.Range(f'C{i}').Value}")
    create_outlook_email(i)

Processing PBACO Patient Activity to Leadership Template.oft
Processing NHNP Patient Activity to Leadership Template.oft
Processing Empire Patient Activity to Leadership Template.oft


In [79]:
wb.Close(SaveChanges=False)
excel_app.Quit()

com_error: (-2147417848, 'The object invoked has disconnected from its clients.', None, None)